## import libraries

In [ ]:
import asyncio
import math
from math import log, sin, cos, tan, exp, sqrt, pi
import time
from random import randrange
import torch
import numpy as np
from classroom import Classroom
from classroom import Student
from classroom import BytesDataset
from classroom import GutenbergSnippetsDataset
from classroom import GutenbergBitsDataset
from classroom import GutenbergGPT2Dataset
from classroom import MLPLM, MyLM, ABPCNLM
from classroom import TransformerLM
from classroom import AdamW
from classroom import Sonny
from classroom import Floyd
from classroom import Plot
from classroom import Fun
from classroom import Count
from classroom import Sum
from classroom import Diff
from classroom import Log2Sum
from classroom import KalmanFilter1D
from classroom import MedianFilter
from classroom import TwoWindowFilter
from classroom import numel
from classroom import utf8decode, utf8encode, gpt2decode, gpt2encode
from classroom import utf8bitsdecode, utf8bitsencode
from pathlib import Path
import numba

## initialize model

In [ ]:
if True:
    path = '2021-12-06-0645.pt'

In [ ]:
if True:
    model = torch.load(path).to('cuda')

In [ ]:
if True:
    model = (
        MLPLM(
            n_vocab_in=50257,
            n_ctx=512,
            d_model=16,
            d_hidden=2048,
            nonlinearity="GELU",
            n_vocab_out=50257).to('cuda'))

In [ ]:
if False:
    model = (
        MyLM(
            n_vocab_in=50257,
            n_ctx=65,
            d_model=64,
            n_layers=1,
            d_hidden=256,
            nonlinearity="GELU",
            p_dropout=0.00,
            n_vocab_out=50257).to('cuda'))

In [ ]:
if False:
    model = (
        ABPCNLM(
            n_vocab_in=50257,
            n_ctx=128,
            d_model=8,
            n_layers=4,
            d_hidden=2048,
            nonlinearity="GELU",
            p_dropout=0.0,
            n_vocab_out=50257).to('cuda'))

In [ ]:
if False:
    model = (
        TransformerLM(
            n_vocab_in=50257,
            n_vocab_out=50257,
            n_ctx=1024,
            d_model=256,
            d_k=16,
            d_v=16,
            n_heads=16,
            d_hidden=512,
            n_layers=3,
            p_dropout_embedding=0,
            p_dropout_attn_mat=0,
            p_dropout_attn_out=0,
            p_dropout_mlp=0).to('cuda'))

In [ ]:
numel(model)

## initialize student

In [ ]:
optimizer = AdamW(parameters=model.named_parameters())
dataset = GutenbergGPT2Dataset()
batch_size = None
example_length = model.n_ctx + 1

student= Student(
    model=model,
    optimizer=optimizer,
    dataset=dataset,
    batch_size=batch_size,
    example_length=example_length)

## schedule hyperparameters

In [ ]:
student.batch_size = 256
student.example_length = 513
for (idx, (pn, p)) in enumerate(student.model.named_parameters()):
    batch_multiplier = 10
    lr_base = 1e-5
    warm_up = 100
    lr = lambda n: 0 if n < warm_up else lr_base *(1 + (n%100))/100
    student.optimizer.state[pn]["lr"]           = lambda n: lr(n)
    student.optimizer.state[pn]["beta1"]        = lambda n: 0.9
    student.optimizer.state[pn]["beta2"]        = lambda n: 0.999
    student.optimizer.state[pn]["weight_decay"] = lambda n: 0.001
    student.optimizer.state[pn]["update"]       = lambda n: (n < warm_up) or (n%batch_multiplier == 0)

## test a single iteration

In [ ]:
# model.language_model.crossentropyloss.crossentropyloss = torch.nn.CrossEntropyLoss(reduction='none')

In [ ]:
# student.study()

## initialize baseline

In [ ]:
if True:
    student.reset_baseline()
    n_of_last_baseline = len(student.times)
    t_start = time.time()
    t_of_last_baseline = 0

## start training

In [ ]:
import asyncio
async def train(student):
    while True:
        student.study()
        await asyncio.sleep(1e-4)

In [ ]:
training_task = asyncio.create_task(train(student))

In [ ]:
training_task

## autocomplete

In [ ]:
def autocomplete(model, prompt=None, n_generate=512,
                     n_ctx=None, temp=1.0,
                     encode=None, decode=None, output=None):
    Categorical = torch.distributions.Categorical
    if n_ctx is None:
        n_ctx = model.n_ctx
    if encode is None:
        encode = gpt2encode
    if decode is None:
        decode = gpt2decode
    if prompt is None:
        prompt = decode(student.dataset.batch(1, 2*n_ctx, offset=None).tolist()[0])  # kludge
    x = encode(prompt)
    x = x[-n_ctx:]
    prompt = decode(x)
    print(f"=== Prompt ===\n{prompt}\n=== Autocompletion ===\n")

    def sampler(x):
        x = list(x)
        for _ in range(n_generate):
            probs = model.inference(torch.tensor(x, dtype=torch.long, device="cuda").unsqueeze(0)).view(-1)[-model.n_vocab_out:]
            if temp > 0:
                y = Categorical(probs=probs**(1.0/temp)).sample().item()
            else:
                y = torch.argmax(probs).item()
            x = (x + [y])[-n_ctx:]
            if output is not None:
                output.append(y)
            yield y
    result = decode(list(sampler(x)))
    print(result)


When first the opposition of fact and ideal grows fully visible, a
spirit of fiery revolt, of fierce hatred of the gods, seems necessary
to the assertion of freedom. To defy with Promethean constancy a hostile
universe, to keep its evil always in view, always actively hated, to
refuse no pain that the malice of Power can invent, appears to be the
duty of all who will not bow before the inevitable. But indignation is
still a bondage, for it compels our thoughts to be occupied with an evil
world; and in the fierceness of desire from which rebellion springs there
is a kind of self-assertion which it is necessary for the wise to overcome.
Indignation is a submission of our thoughts, but not of our desires; the
Stoic freedom in which wisdom consists is found in the submission of our
desires, but not of our thoughts. From the submission of our desires springs
the virtue of resignation; from the freedom of our thoughts springs the whole
world of art and philosophy, and the vision of beauty by which, at last, we
half reconquer the reluctant world.

When first the opposition of fact and ideal grows fully visible, a spirit
of fiery revolt, of fierce hatred of the gods, seems necessary to the
assertion of freedom. To defy with Promethean constancy a hostile universe,
to keep its evil always in view, always actively hated, to refuse no pain
that the malice of Power can invent, appears to be the duty of all who will
not bow before the inevitable. But indignation is still a bondage, for it
compels our thoughts to be occupied with an evil world; and in the fierceness
of desire from which rebellion springs there is a kind of self-assertion
which it is necessary for the wise to overcome. Indignation is a submission
of our thoughts, but not of our desires; the Stoic freedom in which wisdom
consists is found in the submission of our desires, but not of our thoughts.
From the submission of our desires springs the virtue of resignation; from
the freedom of our thoughts springs the whole world of art and philosophy,
and the vision of beauty by which, at last, we half reconquer the reluctant
world.
...

We are of the sun and the moon and the stars.
The power manifested in the mind of God is projected

In [ ]:
%%time
autocomplete(model=student.model, temp=1.0, n_ctx=512, n_generate=512)

## plots

In [ ]:
import time
plot_data = {}
lag = 4096*4
X = Fun(Log2Sum(), student.times)
Y = Fun(TwoWindowFilter(lag=lag), student.grades)
Z = Fun(TwoWindowFilter(lag=lag), student.baseline_grades)
plot_data.update({f"grades": (X, Y)})
plot_data.update({f"baseline": (X, Z)})
Plot(**plot_data)

In [ ]:
import time
plot_data_2 = {}
lag = 1000
X = Fun(Count(), student.times)
Y = Fun(lambda x, y: x - y, student.grades, student.baseline_grades)
Y = Fun(TwoWindowFilter(lag=lag), Y.output, aux=Y)
plot_data_2.update({f"improvement": (X, Y)})
Plot(**plot_data_2)

## stats

In [ ]:
import time

n = len(student.times)-1
t = time.time() - t_start
dn = n - n_of_last_baseline
dt = t - t_of_last_baseline

N = min(dn, 10000)
y = np.mean(np.array(student.grades[n-N:n]))
y0 = np.mean(np.array(student.baseline_grades[n-N:n]))
dy = (y - y0)
z = y0 + dy

lyles_constant = (9115131782/2)/14818489608 * log(50257)/log(256)
utf8grade = lambda x: 1 - (1 - x)*lyles_constant
bpc = (1-utf8grade(z))*8
message = '\n'.join([
    f"bpc                   = {int(bpc*1e6)/1e6}",
    f"batch_size            = {student.batch_size}",
    f"example_length        = {student.example_length}",
    f"100*z                 = {int(z*1e6)/1e4}",
    f"n                     = {n} steps",
    f"t                     = {int(t)} seconds",
    f"n_of_last_baseline    = {n_of_last_baseline} steps",
    f"t_of_last_baseline    = {int(t_of_last_baseline)} seconds",
    f"steps per second      = {dn/dt}",
    f"y0                    = {int(y0*1e6)/1e6}",
    f"dy                    = {int(dy*1e6)}e-06",
    f"dn                    = {dn}",
    f"dt                    = {dt}",
    f"dy/dn                 = {dy/dn}",
    f"dy/dt                 = {int(1e9 * dy/dt)}e-9 per second",
    f"bpcps                 = {int(3600*24*1e3 * dy/dt * (9115131782/2)/14818489608 * log(50257))}e-3 per day",
    f"time for 100%         = {(10.0)/(dy/dt)//36/1000} hours",
])
print(message)

In [ ]:
if True:
    student.reset_baseline()
    n_of_last_baseline = len(student.times)-1
    t_of_last_baseline = time.time() - t_start

## save

In [ ]:
torch.save(student.model, f=path)

In [ ]:
import asyncio
async def autosave():
    while True:
        await asyncio.sleep(3600)
        torch.save(student.model, f='autosave.pt')
task = asyncio.create_task(autosave())


In [ ]:
chars_per_token = 

In [ ]:
lyles_constant = (9115131782/2)/14818489608 * log(50257)/log(256)
lyles_constant

In [ ]:
utf8grade = lambda x: 1 - (1 - x)*lyles_constant
grade = .604 # .49 # .7343
print(f"gpt2 grade = {grade}, utf8 grade = {utf8grade(grade)}, bpc = {(1-utf8grade(grade))*8}")

## parameter histograms

In [ ]:
histograms = []
for (idx, (pn, p)) in enumerate(student.model.named_parameters()):
    with torch.no_grad():
        print(idx, pn, torch.sqrt(torch.var(p)).item())
        Y, X = np.histogram(p.detach().cpu().numpy(), bins=int(sqrt(torch.numel(p))), density=True)
        print(X.shape, Y.shape)
        histograms.append(Plot(**{f"hist-{idx}": (X.tolist(), Y.tolist())}))

In [ ]:
histograms[4] # 3 7 9 13 15 21 43

## batch-level grade histogram

In [ ]:
Y, X = np.histogram(student.grades[-5000:], bins=256, range=(0,1.0), density=True)
V, U = np.histogram(student.baseline_grades[-5000:], bins=256, range=(0,1.0), density=True)
Plot(**{f"grade-hist": (X, Y), "baseline": (U, V)})

In [ ]:
model.n_ctx, model.d_model, model.d_hidden, model.n_layers

## example-level grade histogram

In [ ]:
def get_graded_examples():
    result = []
    for batch_idx in range(16):
        print(f"batch_idx = {batch_idx}/256")
        x = student.dataset.batch(student.batch_size, student.example_length)
        print(f"orig {x.shape}")
        with torch.no_grad():
            y = student.model(x)
            y = 1.0 - y.cpu().numpy()
            result.append(y)
    data = np.concatenate(result, axis=0)
    result = data.tolist()
    return result

In [ ]:
graded_examples = get_graded_examples()

In [ ]:
len(graded_examples)

In [ ]:
sum(x for x in graded_examples)/len(graded_examples)

In [ ]:
R = (0, 1)
def XYFor(k):
    es = graded_examples
    bins = int(sqrt(len(es)))
    Y, X = np.histogram(es, bins=bins, range=R, density=True)
    return (X, Y)
Plot(**{f"examples-hist-{k}": XYFor(k) for k in [1]})

In [ ]:
ord(' ')

In [ ]:
np.mean(example_grades)

In [ ]:
(1 - 0.7870894884999871)*8

In [ ]:
(1 - 0.8)*8

In [ ]:
(1 - 0.9)*8

In [ ]:
x = np.array([[1,2],[3,4]],dtype=np.uint8)

In [ ]:
np.unpackbits(x, axis=1)

In [ ]:
14818489608/(9115131782/2)

In [ ]:
2.0*3.25

In [3]:
prompt = """
To Dr. Faustus in his study Mephistopheles told the history of the Creation, saying:
"The endless praises of the choirs of angels had begun to grow wearisome; for, after all, did he not deserve their praise? Had he not given them endless joy? Would it not be more amusing to obtain undeserved praise, to be worshipped by beings whom he tortured? He smiled inwardly, and resolved that the great drama should be performed.

"For countless ages the hot nebula whirled aimlessly through space. At length it began to take shape, the central mass threw off planets, the planets cooled, boiling seas and burning mountains heaved and tossed, from black masses of cloud hot sheets of rain deluged the barely solid crust. And now the first germ of life grew in the depths of the ocean, and developed rapidly in the fructifying warmth into vast forest trees, huge ferns springing from the damp mould, sea monsters breeding, fighting, devouring, and passing away. And from the monsters, as the play unfolded itself, Man was born, with the power of thought, the knowledge of good and evil, and the cruel thirst for worship. And Man saw that all is passing in this mad, monstrous world, that all is struggling to snatch, at any cost, a few brief moments of life before Death's inexorable decree. And Man said: `There is a hidden purpose, could we but fathom it, and the purpose is good; for we must reverence something, and in the visible world there is nothing worthy of reverence.' And Man stood aside from the struggle, resolving that God intended harmony to come out of chaos by human efforts. And when he followed the instincts which God had transmitted to him from his ancestry of beasts of prey, he called it Sin, and asked God to forgive him. But he doubted whether he could be justly forgiven, until he invented a divine Plan by which God's wrath was to have been appeased. And seeing the present was bad, he made it yet worse, that thereby the future might be better. And he gave God thanks for the strength that enabled him to forgo even the joys that were possible. And God smiled; and when he saw that Man had become perfect in renunciation and worship, he sent another sun through the sky, which crashed into Man's sun; and all returned again to nebula.

"`Yes,' he murmured, `it was a good play; I will have it performed again.'"

Such, in outline, but even more purposeless, more void of meaning, is the world which Science presents for our belief. Amid such a world, if anywhere, our ideals henceforward must find a home. That Man is the product of causes which had no prevision of the end they were achieving; that his origin, his growth, his hopes and fears, his loves and his beliefs, are but the outcome of accidental collocations of atoms; that no fire, no heroism, no intensity of thought and feeling, can preserve an individual life beyond the grave; that all the labours of the ages, all the devotion, all the inspiration, all the noonday brightness of human genius, are destined to extinction in the vast death of the solar system, and that the whole temple of Man's achievement must inevitably be buried beneath the debris of a universe in ruins--all these things, if not quite beyond dispute, are yet so nearly certain, that no philosophy which rejects them can hope to stand. Only within the scaffolding of these truths, only on the firm foundation of unyielding despair, can the soul's habitation henceforth be safely built.

How, in such an alien and inhuman world, can so powerless a creature as Man preserve his aspirations untarnished? A strange mystery it is that Nature, omnipotent but blind, in the revolutions of her secular hurryings through the abysses of space, has brought forth at last a child, subject still to her power, but gifted with sight, with knowledge of good and evil, with the capacity of judging all the works of his unthinking Mother. In spite of Death, the mark and seal of the parental control, Man is yet free, during his brief years, to examine, to criticise, to know, and in imagination to create. To him alone, in the world with which he is acquainted, this freedom belongs; and in this lies his superiority to the resistless forces that control his outward life.

The savage, like ourselves, feels the oppression of his impotence before the powers of Nature; but having in himself nothing that he respects more than Power, he is willing to prostrate himself before his gods, without inquiring whether they are worthy of his worship. Pathetic and very terrible is the long history of cruelty and torture, of degradation and human sacrifice, endured in the hope of placating the jealous gods: surely, the trembling believer thinks, when what is most precious has been freely given, their lust for blood must be appeased, and more will not be required. The religion of Moloch--as such creeds may be generically called--is in essence the cringing submission of the slave, who dare not, even in his heart, allow the thought that his master deserves no adulation. Since the independence of ideals is not yet acknowledged, Power may be freely worshipped, and receive an unlimited respect, despite its wanton infliction of pain.

But gradually, as morality grows bolder, the claim of the ideal world begins to be felt; and worship, if it is not to cease, must be given to gods of another kind than those created by the savage. Some, though they feel the demands of the ideal, will still consciously reject them, still urging that naked Power is worthy of worship. Such is the attitude inculcated in God's answer to Job out of the whirlwind: the divine power and knowledge are paraded, but of the divine goodness there is no hint. Such also is the attitude of those who, in our own day, base their morality upon the struggle for survival, maintaining that the survivors are necessarily the fittest. But others, not content with an answer so repugnant to the moral sense, will adopt the position which we have become accustomed to regard as specially religious, maintaining that, in some hidden manner, the world of fact is really harmonious with the world of ideals. Thus Man creates God, all-powerful and all-good, the mystic unity of what is and what should be.

But the world of fact, after all, is not good; and, in submitting our judgment to it, there is an element of slavishness from which our thoughts must be purged. For in all things it is well to exalt the dignity of Man, by freeing him as far as possible from the tyranny of non-human Power. When we have realised that Power is largely bad, that man, with his knowledge of good and evil, is but a helpless atom in a world which has no such knowledge, the choice is again presented to us: Shall we worship Force, or shall we worship Goodness? Shall our God exist and be evil, or shall he be recognised as the creation of our own conscience?

The answer to this question is very momentous, and affects profoundly our whole morality. The worship of Force, to which Carlyle and Nietzsche and the creed of Militarism have accustomed us, is the result of failure to maintain our own ideals against a hostile universe: it is itself a prostrate submission to evil, a sacrifice of our best to Moloch. If strength indeed is to be respected, let us respect rather the strength of those who refuse that false "recognition of facts" which fails to recognise that facts are often bad. Let us admit that, in the world we know, there are many things that would be better otherwise, and that the ideals to which we do and must adhere are not realised in the realm of matter. Let us preserve our respect for truth, for beauty, for the ideal of perfection which life does not permit us to attain, though none of these things meet with the approval of the unconscious universe. If Power is bad, as it seems to be, let us reject it from our hearts. In this lies Man's true freedom: in determination to worship only the God created by our own love of the good, to respect only the heaven which inspires the insight of our best moments. In action, in desire, we must submit perpetually to the tyranny of outside forces; but in thought, in aspiration, we are free, free from our fellow-men, free from the petty planet on which our bodies impotently crawl, free even, while we live, from the tyranny of death. Let us learn, then, that energy of faith which enables us to live constantly in the vision of the good; and let us descend, in action, into the world of fact, with that vision always before us.

When first the opposition of fact and ideal grows fully visible, a spirit of fiery revolt, of fierce hatred of the gods, seems necessary to the assertion of freedom. To defy with Promethean constancy a hostile universe, to keep its evil always in view, always actively hated, to refuse no pain that the malice of Power can invent, appears to be the duty of all who will not bow before the inevitable. But indignation is still a bondage, for it compels our thoughts to be occupied with an evil world; and in the fierceness of desire from which rebellion springs there is a kind of self-assertion which it is necessary for the wise to overcome. Indignation is a submission of our thoughts, but not of our desires; the Stoic freedom in which wisdom consists is found in the submission of our desires, but not of our thoughts. From the submission of our desires springs the virtue of resignation; from the freedom of our thoughts springs the whole world of art and philosophy, and the vision of beauty by which, at last, we half reconquer the reluctant world.
"""

In [5]:
print(prompt)


To Dr. Faustus in his study Mephistopheles told the history of the Creation, saying:
"The endless praises of the choirs of angels had begun to grow wearisome; for, after all, did he not deserve their praise? Had he not given them endless joy? Would it not be more amusing to obtain undeserved praise, to be worshipped by beings whom he tortured? He smiled inwardly, and resolved that the great drama should be performed.

"For countless ages the hot nebula whirled aimlessly through space. At length it began to take shape, the central mass threw off planets, the planets cooled, boiling seas and burning mountains heaved and tossed, from black masses of cloud hot sheets of rain deluged the barely solid crust. And now the first germ of life grew in the depths of the ocean, and developed rapidly in the fructifying warmth into vast forest trees, huge ferns springing from the damp mould, sea monsters breeding, fighting, devouring, and passing away. And from the monsters, as the play unfolded its